# Dropout Regularization

* In Visible Layer
* In Hidden Layer

### In Visible Layer

In [12]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dropout
from keras.constraints import maxnorm
from keras.optimizers import SGD


seed = 7
numpy.random.seed(seed)

dataframe = pandas.read_csv("03-sonar.csv", delimiter=',', header=None)
dataset = dataframe.values

X = dataset[:,0:60].astype(float)
y = dataset[:,60]

encoder = LabelEncoder()
encoder.fit(Y)
encoded_y = encoder.transform(y)

def create_baseline(optimizer='adam', init='normal'):
    model = Sequential()
    model.add(Dropout(0.2, input_shape=(60,)))
    model.add(Dense(60, input_dim=60, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dense(30, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dense(1, activation='sigmoid'))
    sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model
    
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=300, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print("Accuracy After Visible Dropout Reg: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Accuracy After Visible Dropout Reg: 82.16% (5.13%)


### In Hidden Layer

In [13]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dropout
from keras.constraints import maxnorm
from keras.optimizers import SGD


seed = 7
numpy.random.seed(seed)

dataframe = pandas.read_csv("03-sonar.csv", delimiter=',', header=None)
dataset = dataframe.values

X = dataset[:,0:60].astype(float)
y = dataset[:,60]

encoder = LabelEncoder()
encoder.fit(Y)
encoded_y = encoder.transform(y)

def create_baseline():
    model = Sequential()
    model.add(Dense(60, input_dim=60, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(30, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model
    
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=300, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print("Accuracy After Hidden Dropout Reg: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Accuracy After Hidden Dropout Reg: 78.85% (7.83%)
